## Kernel: `Python 3`

# Day 2 - Tutorial 2

The objective of this tutorial is to learn how to create type well curves using production data

In [0]:
# Import required libraries

import pandas as pd
from IPython.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt

import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [0]:
# Read and display the completion data 

mydataset = dataiku.Dataset("completion_data")
completion = mydataset.get_dataframe()

completion

In [0]:
# Create a function to display a heading on the dataframe when working in chain mode (method chaining)

def show(df_, title=None):
    if title:
        display(HTML(f"<h2 >{title}</h2>"))
        display(df_)
    return df_


In [0]:
# Create a function to store the resulting dataframe when working in chain mode (method chaining)

def get_var(df_, var_name):
    globals()[var_name] = df_
    return df_


In [0]:
# Create a function to edit the completion dataset as follows:

def prep_completion(df_):
    '''
    this function does the following
    1- Changes columns name to lower case
    2- Replaces space with underscore (_) in column names 
    3- Keep the rows when producing zone= 'zone1'
    4- Removes the rows when fluid column = 'oil'
    :param df_: original data frame
    :return: processed data frame
    '''
    return (df_
            .rename(columns={k: k.lower().replace(' ', '_') for k in df_.columns})
            .query("producing_zone in ['zone1']")
            .query("fluid not in ['oil']")
            .pipe(show, title="keeping Gas Only")
            .pipe(get_var, 'new_cols') 
            )

# 'new_cols' is a dataframe, so once we apply the fuction, we can inspect it! 

In [0]:
# Now let's apply the prep_completion function to the completion dataset



In [0]:
# Read and display the production data 

mydataset = dataiku.Dataset("production_data")
production = mydataset.get_dataframe()


# Make sure that the column 'Prod Date' is treated as date

production["Prod Date"]= pd.to_datetime(production["Prod Date"])

production

In [0]:
# Create a function to modify the production dataset following the instrutions below:

def prep_production(df_):
    '''
    this function does the following:
    1- lower case the columns name
    2- Replace spaces by _
    3- set 'prod_date' as index
    :param df_: original data frame
    :return: processed data frame
    '''
    return (df_
            .rename(columns={k: k.lower().replace(' ', '_') for k in df_.columns})
            .set_index("prod_date")
            .pipe(show,title="Edited Prod")
            .pipe(get_var, 'new_cols')  
            )


In [0]:
# Now let's apply the 'prep_completion' function to the production dataset



In [0]:
# Use the fuction .groupby to plot the production data for all the wells in a single graph

plt.figure(figsize=(15, 6))
production.groupby("unformatted_uwi")["monthly_gas_(e3m3)"].plot(marker='.', markersize=5, legend=True, 
                                                         title='Monthly Gas Production')

plt.xlabel('Production Date')
plt.ylabel('Monthly Gas Production (e3m3)')


In [0]:
# Merge the completion and production datasets and make the following changes to the new data set using method chaining

#1 Reset the current index
#2 Rename the column 'unformatted_uwi' to 'uwi'
#3 Merge the completion and production datasets based on the 'uwi' column
#4 Sort the dataframe based on the 'prod_date' column
#5 Create a new column named 'normalized_date' to number each production entry for a well -> (0 - max # of prod. rows)
#6 create a new column named 'monthly_gas_per_stimulated_length' and compute 'monthly_gas_(e3m3)' / ddf.stimulated_length
#7 Drop null values (NaN) from the 'monthly_gas_per_stimulated_length' column
#8 Set the 'normalized_date' column as index


prod_comp = (production
        .reset_index() 
        .rename(columns={"unformatted_uwi": "uwi"})
        .merge(completion[["uwi", "stimulated_length"]], how="left", left_on="uwi", right_on="uwi") 
        .sort_values(by=["uwi", "prod_date"])
        .assign(normalized_date= lambda df_: df_.groupby("uwi").cumcount().values,
                monthly_gas_per_stimulated_length=lambda ddf: ddf['monthly_gas_(e3m3)'] / ddf['stimulated_length'])
        .dropna(subset=["monthly_gas_per_stimulated_length"]) 
        .set_index("normalized_date") 
)

In [0]:
# Print the 'prod_comp' dataframe to visualize the changes made



In [0]:
# Now let's plot the monthly gas production versus normalized_date 



## Create Percentiles

In [0]:
# Create a function to define custom percentiles

def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)

    percentile_.__name__ = 'P_ %s' % n
    return percentile_


In [0]:
# Let's review what the percentile (n) function does:

prod_comp.groupby('normalized_date').agg([percentile(10), 
                                          percentile(25), 
                                          percentile(50), 
                                          percentile(75), 
                                          percentile(90)])["monthly_gas_per_stimulated_length"]

# This funtion computes percentile values (10, 25, 50, 75, 90) for the total monthly gas production of the four wells 

In [0]:
# First let's create a custom plotting function

def format_plot(ax=None, label=""):
    ax.set_ylabel(label, fontdict={'fontsize': 10})
    ax.set_xlabel('Month #', fontdict={'fontsize': 10}) 
    plt.grid(b=True, which='major', color='gray', linestyle='-')
    plt.grid(b=True, which='minor', linestyle='--')
    plt.minorticks_on()
    plt.tight_layout()
    plt.show()


In [0]:
# Create a function to generate two plots to display the total production for 5 years (60 months)

# The first plot is monthly_gas_per_stimulated_length versus Month # ('normalized_date')
# The second plot is the 'Cum Monthly Gas/stimulated length' versus Month # ('normalized_date')


def plot_type_wells(df: pd.DataFrame = "",
                    col="",
                    col_label="",
                    cum_col_label=""):
    
    ax = prod_comp.groupby('normalized_date').agg([percentile(10), 
                                                   percentile(25), 
                                                   percentile(50), 
                                                   percentile(75), 
                                                   percentile(90)])[col].head(60).plot(fontsize=14, 
                                                                                       linewidth=3)

    format_plot(ax=ax, label=col_label)
    
    
    ax = prod_comp.groupby('normalized_date').agg([percentile(10), 
                                                   percentile(25),
                                                   percentile(50),
                                                   percentile(75),
                                                   percentile(90)])[col].head(60).cumsum().plot(fontsize=14,
                                                                                                linewidth=3)
    
    format_plot(ax=ax, label=cum_col_label)

    
# Note that the .cumsum() function was agregated to be able to generate the second plot  

In [0]:
# Apply the 'plot_type_wells' function to display the monthly and cummulative normalized production plots

plot_type_wells(df=prod_comp, 
                col="monthly_gas_per_stimulated_length",
                col_label="Monthly Gas/stimulated length (E3m3)",
                cum_col_label="Cum Monthly Gas/stimulated length (E3m3)")